# Exchanges exploration

In [143]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
from pprint import pprint

import bw2analyzer as bwa
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import bw_processing as bwp
import numpy as np

# import ipywidgets as widgets
import pandas as pd
from bw2data.query import Filter, Query
from IPython.display import display
from tqdm import autonotebook

In [68]:
from project_details import EI_DB_NAME, PROJECT_NAME

In [69]:
bd.projects.set_current(PROJECT_NAME)
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

In [70]:
# Is the background database name the same as the one we wrote in `propject_details.py`?
assert EI_DB_NAME in bd.databases

In [71]:
FG_DB_NAME = "asphalt"

In [72]:
db_asphalt = bd.Database(FG_DB_NAME)

In [73]:
pavement_complete_a = db_asphalt.get("DZOAB, A")
pavement_complete_a_PVI = db_asphalt.get("DZOAB, A, PVI")
pavement_complete_b = db_asphalt.get("DZOAB, B")
pavement_complete_b_PVI = db_asphalt.get("DZOAB, B, PVI")

In [74]:
bwa.print_recursive_supply_chain(pavement_complete_a_PVI, max_level=1)

1: 'DZOAB, A, PVI' (kilogram, NL, None)
  1: 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)
  1: 'A1, pavement, materials, A (own material)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (bitumen)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (crushed stone)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (own material)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (crushed sand)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (drip resistant material)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (filler)' (kilogram, NL, None)
  1: 'A3, pavement, production, A' (kilogram, NL, None)
  1: 'A4, pavement, transport to site' (kilogram, NL, None)
  1: 'A5, pavement, construction' (kilogram, NL, None)
  1: 'B, pavement, use (car and HDV)' (kilogram, NL, None)
  1: 'B, pavement, use (HDV+)' (kilogram, NL, None)
  1: 'C1, pavement, demolition' (kilogram, NL, None)
  1:

## Associate exchanges to samples

### Work with the pavement materials

First let's verify that we can use the information already contained in the db.

The excel file for the foreground provides a comments field that gives a generic name of the exchange. This can be linked to the name of the column of the generated samples.

Let's first look at the pavement materials data to create the data structures that we could use later with data packages.

In [76]:
db_asphalt.search('A1, materials (excl. own material)')[0].as_dict()

{'reference product': 'A1, pavement, materials (excl. own material)',
 'code': 'A1_materials_A_excl.ownmaterial',
 'location': 'NL',
 'amount': 1,
 'unit': 'kilogram',
 'original_ConversionDem2FU': 1,
 'name': 'A1, pavement, materials, A (excl. own material)',
 'worksheet name': 'Sheet1',
 'database': 'asphalt',
 'id': 25987}

In [77]:
pavement_mats = bd.get_activity(25987)
print(pavement_mats)

'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)


In [78]:
for e in pavement_mats.technosphere():
    print(e)

Exchange: 0.3 kilogram 'asphalt granulate, free of burden' (kilogram, NL, None) to 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)>
Exchange: 0.0412 kilogram 'bitumen adhesive compound production, hot' (kilogram, RER, None) to 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)>
Exchange: 0.5861000000000001 kilogram 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None) to 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)>
Exchange: 0.0342 kilogram 'gravel production, crushed' (kilogram, RoW, None) to 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)>
Exchange: 0.0021000000000000003 kilogram 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None) to 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)>
Exchange: 0.027 kilogram 'medium filler' (kilogram, NL, None) to 'A1, pavement, materials, A (excl. own material)' 

In [79]:
#detail look to exchanges of interest
[exc for exc in pavement_mats.technosphere()][2].as_dict() 

{'name': 'crushed stone, from quarry in Europe, excluding transport to the Netherlands',
 'reference product': 'crushed stone',
 'location': 'NL',
 'amount': 0.5861000000000001,
 'unit': 'kilogram',
 'database': 'ecoinvent-391-cutoff',
 'type': 'technosphere',
 'categories': ('(unknown)',),
 'comments': 'crushed stone',
 'uncertainty_type': 2,
 'loc': -0.5342648554997534,
 'scale': 0.03741657386773941,
 'input': ('asphalt', 'crushed_stone'),
 'output': ('asphalt', 'A1_materials_A_excl.ownmaterial')}

The exchanges include a "comments" field

In [80]:
for e in pavement_mats.technosphere():
    if "comments" in e.as_dict():
        # print(e["comments"])
        pprint(e["comments"])

'rap'
'bitumen'
'crushed stone'
'crushed sand'
'drip resistant material'
'filler'


In [81]:
def exchange_coords(exchange):
    """create a tuple with the numerical ids of the input and output activites of an exchange."""
    input_activity = bd.get_activity(exchange["input"])
    output_activity = bd.get_activity(exchange["output"])
    return (input_activity.id, output_activity.id)

In [82]:
# We can create tuples of coords for every non "production" exchange of an activity like this.
for e in pavement_mats.technosphere():
    print(exchange_coords(e))

(25975, 25987)
(8553, 25987)
(25976, 25987)
(17503, 25987)
(25974, 25987)
(25977, 25987)


### Parse all the pre-generated samples.

In [107]:
samples_df = pd.read_excel(
    "Final_samples.xlsx", sheet_name=pavement_complete_a["name"], dtype=float, index_col=0
)
samples_df.head()

,A1_bitumen,A1_crushedsand,A1_crushedstone3,A1_asphaltgranulate,A1_ownmaterial,A1_mediumfiller,A1_dripresistantmaterial,A2_bitumen,A2_crushedsand_t,A2_crushedsand_iv,...,A3_electricity,A3_diesel,A4_distance_EURO5,A4_distance_EURO6,A5_construction,C1_removal,C2_distance_EURO5,C2_distance_EURO6,C3_craneandshovel,C3_breaker
0,0.042558,0.035116,0.591192,0.299967,0.009732,0.025648,0.002002,0.009872,0.000902,0.028033,...,0.005639,0.003824,0.023854,0.007951,0.009716,0.024317,0.035624,0.011875,0.006003,0.005458
1,0.039691,0.032146,0.577359,0.327039,0.009244,0.028904,0.002002,0.011234,0.000900,0.014003,...,0.006124,0.003739,0.036901,0.012300,0.010399,0.025039,0.022755,0.007585,0.006007,0.006206
2,0.040694,0.035285,0.579170,0.300157,0.009573,0.027313,0.002013,0.012900,0.001119,0.046039,...,0.005531,0.003930,0.023517,0.007839,0.010004,0.023950,0.023741,0.007914,0.005880,0.006016
3,0.040881,0.035088,0.572281,0.288649,0.009698,0.027617,0.002028,0.012794,0.000880,0.019909,...,0.005544,0.003700,0.034301,0.011434,0.009605,0.023829,0.019934,0.006645,0.005800,0.005765
4,0.040095,0.035370,0.572222,0.332977,0.009691,0.025722,0.002132,0.011271,0.001263,0.027920,...,0.006050,0.003761,0.044652,0.014884,0.010502,0.024140,0.048939,0.016313,0.005469,0.005981


In [108]:
samples_df.columns

Index(['A1_bitumen', 'A1_crushedsand', 'A1_crushedstone3',
       'A1_asphaltgranulate', 'A1_ownmaterial', 'A1_mediumfiller',
       'A1_dripresistantmaterial', 'A2_bitumen', 'A2_crushedsand_t',
       'A2_crushedsand_iv', 'A2_crushedstone3_t', 'A2_crushedstone3_iv',
       'A2_crushedstone3_sv', 'A2_ownmaterial_t', 'A2_ownmaterial_iv',
       'A2_mediumfiller', 'A2_dripresistantmaterial', 'A3_naturalgas',
       'A3_electricity', 'A3_diesel', 'A4_distance_EURO5', 'A4_distance_EURO6',
       'A5_construction', 'C1_removal', 'C2_distance_EURO5',
       'C2_distance_EURO6', 'C3_craneandshovel', 'C3_breaker'],
      dtype='object')

Each sample column is meant to be associated to one phase (A1, A2, etc.) and an inputs of a pavement exchange.
We can map the comment of the exchange to an existing samples column name roughly like this:


In [85]:
def sample_name_for_exchange(exchange, phase):
    """Return the sample name to use for this exchange."""
    all_names_mapping = {
        "A1": {
            "bitumen": "A1_bitumen",
            "crushed sand": "A1_crushedsand",
            "crushed stone": "A1_crushedstone3",
            "rap": "A1_asphaltgranulate",
            "own material": "A1_ownmaterial",
            "filler": "A1_mediumfiller",
            "drip resistant material": "A1_dripresistantmaterial",
        },
        "A2": {
            "bitumen": "A2_bitumen",
            "filler": "A2_mediumfiller",
            "drip resistant material": "A2_dripresistantmaterial",
        },
    }

    if "A1" == phase:
        names_mapping = all_names_mapping[phase]
        col_name = names_mapping[exchange["comments"]]
    if "A2" == phase:
        names_mapping = all_names_mapping[phase]
        if exchange["comments"] in names_mapping:
            return names_mapping[exchange["comments"]]
        else:
            input_activity = bd.get_activity(exchange["input"])
            comments = exchange["comments"]
            var_name = exchange["comments"].replace(" ", "")
            if var_name == "crushedstone":
                var_name = var_name + "3"
            ref_prod = input_activity["reference product"]
            if ref_prod == "transport, freight, sea, ferry":
                suffix = "sv"
            if ref_prod == "transport, freight, inland waterways, barge":
                suffix = "iv"
            if ref_prod == "transport, freight, lorry, unspecified":
                suffix = "t"
            col_name = f"{phase}_{var_name}_{suffix}"
    if "A3" == phase:
        input_activity = bd.get_activity(exchange["input"])
        ref_prod = input_activity["reference product"]
        if ref_prod == "electricity, low voltage":
            col_name = f"{phase}_electricity"
        elif ref_prod == "heat, district or industrial, natural gas":
            col_name = f"{phase}_naturalgas"
        elif ref_prod == "diesel, burned in building machine":
            col_name = f"{phase}_diesel"
    if "A4" == phase or "C2" == phase:
        input_activity = bd.get_activity(exchange["input"])
        comments = exchange["comments"]
        if comments == "EURO5":
            col_name = f"{phase}_distance_EURO5"
        elif comments == "EURO6":
            col_name = f"{phase}_distance_EURO6"
    if "A5" == phase:
        col_name = f"{phase}_construction"
    if "B" == phase:
        input_activity = bd.get_activity(exchange["input"])
        comments = exchange["comments"]
        if comments == "car":
            col_name = f"{phase}_cars_avg"
        elif comments == "HDV":
            col_name = f"{phase}_HDV_avg"
        elif comments == "HDV+":
            col_name = f"{phase}_HDVtrailer_avg"
    if "C1" == phase:
        col_name = f"{phase}_removal"
    if "C3" == phase:
        input_activity = bd.get_activity(exchange["input"])
        comments = exchange["comments"]
        if comments == "shovel":
            col_name = f"{phase}_craneandshovel"
        elif comments == "breaking":
            col_name = f"{phase}_breaker"

    return col_name

In [128]:
def build_coords_sample(activity, samples_df):
    """Build the coords + sample dictionnary for all phases of activity.

    The activities are the top level "pavement complete" producing ones:

    [0]: DZOAB, A, PVI (pavement, complete, NL)
    [1]: DZOAB, B, PVI (pavement, complete, NL)
    [2]: DZOAB, B (pavement, complete, NL)
    [3]: DZOAB, A (pavement, complete, NL)

    """
    # Make sure we have a valid activity to start with
    assert "pavement, complete" == activity["reference product"]
    coords_samples_map = {}

    # the phases are for example: A1, pavement, materials, A
    # We must create the coords_sample for the inputs of each "phase"
    phase_exchanges = [input for input in activity.technosphere()]
    counter = 0
    for phase_exchange in phase_exchanges:
        input_activity = bd.get_activity(phase_exchange["input"])
        # pprint(input_activity)
        # mapping = build_mapping(input_activity)
        # print(mapping)
        # Extract the "A1", "A2", etc. substring from the phase name
        current_phase = input_activity["name"].split(",")[0]
        for exchange in input_activity.technosphere():
            col_name = sample_name_for_exchange(exchange, current_phase)
            coords = exchange_coords(exchange)
            counter += 1
            print(f"{counter}. {exchange}\n\t 👉 {coords} to {col_name}")
            coords_samples_map[exchange_coords(exchange)] = samples_df[
                col_name
            ].values  # numpy.ndarray
    return coords_samples_map

In [129]:
# This is how to build the coords_samples_map for the full pavement activity
coords_samples_map = build_coords_sample(pavement_complete_a, samples_df)

1. Exchange: 0.3 kilogram 'asphalt granulate, free of burden' (kilogram, NL, None) to 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)>
	 👉 (25975, 25987) to A1_asphaltgranulate
2. Exchange: 0.0412 kilogram 'bitumen adhesive compound production, hot' (kilogram, RER, None) to 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)>
	 👉 (8553, 25987) to A1_bitumen
3. Exchange: 0.5861000000000001 kilogram 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None) to 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)>
	 👉 (25976, 25987) to A1_crushedstone3
4. Exchange: 0.0342 kilogram 'gravel production, crushed' (kilogram, RoW, None) to 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)>
	 👉 (17503, 25987) to A1_crushedsand
5. Exchange: 0.0021000000000000003 kilogram 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None) to 'A1, pavement, materia

Here is a dictionnary with the coords + vector with the samples.

In [130]:
counter = 0
for k, v in coords_samples_map.items():
    counter +=1
    print(f"{counter}. {k} --array-> {v}")

1. (25975, 25987) --array-> [0.29996745 0.32703866 0.30015698 ... 0.3049856  0.30737276 0.29226108]
2. (8553, 25987) --array-> [0.0425576  0.03969113 0.04069381 ... 0.04056687 0.03873746 0.04120409]
3. (25976, 25987) --array-> [0.59119169 0.57735947 0.57916985 ... 0.58078443 0.56440685 0.59239116]
4. (17503, 25987) --array-> [0.03511603 0.03214571 0.03528521 ... 0.03615064 0.03500219 0.03637127]
5. (25974, 25987) --array-> [0.00200194 0.00200153 0.00201293 ... 0.00218769 0.00213234 0.00218444]
6. (25977, 25987) --array-> [0.02564781 0.02890423 0.02731286 ... 0.02727033 0.02606585 0.02617468]
7. (25976, 25988) --array-> [0.00973153 0.00924391 0.00957255 ... 0.00958156 0.00904519 0.00980764]
8. (18370, 25990) --array-> [0.0098724  0.01123424 0.01289974 ... 0.00780484 0.00659053 0.00662358]
9. (10822, 25991) --array-> [0.74794219 0.38524552 0.31259779 ... 0.78730368 1.03991455 0.66707487]
10. (9408, 25991) --array-> [0.0294236  0.02355408 0.02403601 ... 0.02609757 0.03371037 0.02409302]
1

In [89]:
# cml_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
cml_method_gwp = (
    "CML v4.8 2016",
    "climate change",
    "global warming potential (GWP100)",
)

## Regular LCA from activity + method

In [132]:
a_lca = bc.LCA({pavement_complete_a: 1}, cml_method_gwp)

In [133]:
a_lca.lci()
a_lca.lcia()
a_lca.score

0.15512695599913062

## LCA using datapackages

In [142]:
bd.prepare_lca_inputs??

Signature:
bd.prepare_lca_inputs(
    demand=None,
    method=None,
    weighting=None,
    normalization=None,
    demands=None,
    remapping=True,
    demand_database_last=True,
)
Source:   
def prepare_lca_inputs(
    demand=None,
    method=None,
    weighting=None,
    normalization=None,
    demands=None,
    remapping=True,
    demand_database_last=True,
):
    """Prepare LCA input arguments in Brightway 2.5 style."""
    if not projects.dataset.data.get("25"):
        raise Brightway2Project(
            "Please use `projects.migrate_project_25` before calculating using Brightway 2.5"
        )

    databases.clean()
    data_objs = []
    remapping_dicts = None

    if demands:
        demand_database_names = [
            db_label for dct in demands for db_label, _ in unpack(dct)
        ]
    elif demand:
        demand_database_names = [db_label for db_label, _ in unpack(demand)]
    else:
        demand_database_names = []

    if demand_database_names:
        database_n

In [97]:
# the following function returns:
# indexed_demand, data_objs, remapping_dicts
# that follow the data package spec
indexed_demand, data_objs, remapping_dicts = bd.prepare_lca_inputs(
    {pavement_complete_a: 1}, cml_method_gwp
)

In [98]:
indexed_demand

{26010: 1}

In [99]:
data_objs

In [135]:
remapping_dicts

{'activity': {25986: ('asphalt', 'A1_materials_A'),
  25987: ('asphalt', 'A1_materials_A_excl.ownmaterial'),
  25988: ('asphalt', 'A1_materials_A_ownmaterial'),
  25978: ('asphalt', 'A1_materials_B'),
  25989: ('asphalt', 'A2_transport_to_plant_A'),
  25990: ('asphalt', 'A2_transport_to_plant_A_bitumen'),
  25993: ('asphalt', 'A2_transport_to_plant_A_crushedsand'),
  25991: ('asphalt', 'A2_transport_to_plant_A_crushedstone'),
  25994: ('asphalt', 'A2_transport_to_plant_A_dripresistantmaterial'),
  25995: ('asphalt', 'A2_transport_to_plant_A_filler'),
  25992: ('asphalt', 'A2_transport_to_plant_A_ownmaterial'),
  25979: ('asphalt', 'A2_transport_to_plant_B'),
  25980: ('asphalt', 'A2_transport_to_plant_B_bitumen'),
  25982: ('asphalt', 'A2_transport_to_plant_B_crushedsand'),
  25981: ('asphalt', 'A2_transport_to_plant_B_crushedstone'),
  25983: ('asphalt', 'A2_transport_to_plant_B_dripresistantmaterial'),
  25984: ('asphalt', 'A2_transport_to_plant_B_filler'),
  25996: ('asphalt', 'A3_p

In [100]:
for i in data_objs:
    print(i.metadata["name"])

biosphere3
ecoinvent-391-cutoff
asphalt
cml-v48-2016cg.231d6e8f8b1c199a47182515eba4032e.zip


In [101]:
dp_lca = bc.LCA(demand=indexed_demand, data_objs=data_objs)
dp_lca.lci()
dp_lca.lcia()
dp_lca.score

0.15512695599913062

In [102]:
biosphere_dp = [dp for dp in data_objs if dp.metadata["name"] == "biosphere3"].pop()
biosphere_dp.metadata["name"]

'biosphere3'

In [103]:
ei_dp = [dp for dp in data_objs if dp.metadata["name"] == EI_DB_NAME].pop()
ei_dp.metadata["name"]

'ecoinvent-391-cutoff'

In [104]:
fg_dp = [dp for dp in data_objs if dp.metadata["name"] == "asphalt"].pop()
fg_dp.metadata["name"]

'asphalt'

In [136]:
pprint(fg_dp.metadata)

{'combinatorial': False,
 'created': '2024-04-02T17:03:07.471365Z',
 'id': '3494d9184720464492206de1a498989d',
 'licenses': [{'name': 'ODC-PDDL-1.0',
               'path': 'http://opendatacommons.org/licenses/pddl/',
               'title': 'Open Data Commons Public Domain Dedication and '
                        'License v1.0'}],
 'name': 'asphalt',
 'profile': 'data-package',
 'resources': [{'category': 'vector',
                'format': 'npy',
                'group': 'asphalt_inventory_geomapping_matrix',
                'kind': 'indices',
                'matrix': 'inv_geomapping_matrix',
                'mediatype': 'application/octet-stream',
                'name': 'asphalt_inventory_geomapping_matrix.indices',
                'nrows': 36,
                'path': 'asphalt_inventory_geomapping_matrix.indices.npy',
                'profile': 'data-resource'},
               {'category': 'vector',
                'format': 'npy',
                'group': 'asphalt_inventory_geoma

In [106]:
dp_correlated = bwp.create_datapackage(sequential=True)

In [111]:
samples_indices = [coord for coord in coords_samples_map.keys()]
pprint(samples_indices)

[(25975, 25987),
 (8553, 25987),
 (25976, 25987),
 (17503, 25987),
 (25974, 25987),
 (25977, 25987),
 (25976, 25988),
 (18370, 25990),
 (10822, 25991),
 (9408, 25991),
 (18370, 25991),
 (9408, 25992),
 (18370, 25992),
 (9408, 25993),
 (18370, 25993),
 (18370, 25994),
 (18370, 25995),
 (25065, 25996),
 (20515, 25996),
 (18987, 25996),
 (16115, 25997),
 (8986, 25997),
 (25948, 25998),
 (25949, 26002),
 (16115, 26003),
 (8986, 26003),
 (18987, 26005),
 (18987, 26006)]


In [139]:
# Prepare the data_array, it must be an array of arrays
samples_values = np.array([coords_samples_map[coord] for coord in samples_indices])
pprint(samples_values)

array([[0.29996745, 0.32703866, 0.30015698, ..., 0.3049856 , 0.30737276,
        0.29226108],
       [0.0425576 , 0.03969113, 0.04069381, ..., 0.04056687, 0.03873746,
        0.04120409],
       [0.59119169, 0.57735947, 0.57916985, ..., 0.58078443, 0.56440685,
        0.59239116],
       ...,
       [0.01187474, 0.00758516, 0.00791365, ..., 0.01155531, 0.01935971,
        0.00464268],
       [0.00600252, 0.00600731, 0.00588028, ..., 0.0056302 , 0.00624567,
        0.00604022],
       [0.00545812, 0.00620634, 0.00601627, ..., 0.00585346, 0.00552031,
        0.00553195]])


In [113]:
dp_correlated.add_persistent_array(
    matrix="technosphere_matrix",
    indices_array=np.array(samples_indices, dtype=bwp.INDICES_DTYPE),
    data_array=samples_values,
    # We flip the signs of the samples
    # to obey bw's convention
    flip_array=np.array([True for _ in range(len(samples_values))]),
)

### Build the new data objects to re-do the LCA

In [114]:
data_objs.append(dp_correlated)

In [115]:
dp_lca = bc.LCA(demand=indexed_demand, data_objs=data_objs, use_arrays=True)
dp_lca.lci()
dp_lca.lcia()
dp_lca.score

0.1824022717061968

In [122]:
scores = []
for _ in autonotebook.tqdm(range(50)):
    next(dp_lca)
    scores.append(dp_lca.score)

100%|█████████████████████████████████████████████████████| 50/50 [04:59<00:00,  5.99s/it]


In [123]:
scores_a = np.array(scores)
scores_a.mean()

0.16493318626543826

In [137]:
scores_a_s = pd.Series(scores_a)
print(scores_a_s)

0     0.162284
1     0.157110
2     0.173419
3     0.148478
4     0.146166
5     0.194811
6     0.155000
7     0.147701
8     0.200901
9     0.186095
10    0.148647
11    0.203662
12    0.178047
13    0.197044
14    0.158938
15    0.181784
16    0.198502
17    0.171664
18    0.150360
19    0.150826
20    0.169061
21    0.135195
22    0.141358
23    0.186688
24    0.140305
25    0.119402
26    0.194544
27    0.137925
28    0.150871
29    0.163052
30    0.166977
31    0.192874
32    0.149546
33    0.134772
34    0.220602
35    0.131615
36    0.162099
37    0.208634
38    0.152460
39    0.150171
40    0.129086
41    0.134996
42    0.171125
43    0.184029
44    0.148167
45    0.281251
46    0.141841
47    0.125202
48    0.164808
49    0.146567
dtype: float64


In [125]:
scores_a_s.describe()

count    50.000000
mean      0.164933
std       0.029402
min       0.119402
25%       0.146850
50%       0.158024
75%       0.183468
max       0.281251
dtype: float64